## Collecting protocol fees across Balancer core pools on all networks
- Spreadsheet as reference: https://docs.google.com/spreadsheets/d/1xwUPpbYq7woVOU9vQ8EB8MY75I-1mauTLyDVwvKUDKo/edit#gid=0
- Collab: https://colab.research.google.com/drive/1vKCvcV5mkL1zwW3565kLSGkBEbt8NsoB?usp=sharing


In [2]:
from dotenv import load_dotenv

load_dotenv()
import datetime

import pandas as pd

from fee_allocator.helpers import get_block_by_ts
from fee_allocator.helpers import get_twap_bpt_price
from fee_allocator.config_file import Chains
from fee_allocator.config_file import WEB3_INSTANCES
from fee_allocator.config_file import REROUTE_CONFIG
from fee_allocator.config_file import FEES_CONSTANTS
from fee_allocator.config_file import CORE_POOLS

MIN_AURA_INCENTIVE = FEES_CONSTANTS['min_aura_incentive']
MIN_INCENTIVE = FEES_CONSTANTS['min_vote_incentive_amount']

TARGET_BLOCKS = {}
timestamp_now = 1697148000
timestamp_2_weeks_ago = timestamp_now - (2 * 7 * 24 * 60 * 60)
datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
datetime_2_weeks_ago = datetime.datetime.fromtimestamp(timestamp_2_weeks_ago)

target_blocks = {}
bpt_twap_prices = {chain.value: {} for chain in Chains}
# Collect BPT prices for 2 weeks ago and now for all relevant pools and chains
for chain in Chains:
    target_blocks[chain.value] = (
        get_block_by_ts(timestamp_now, chain.value),  # Block now
        get_block_by_ts(timestamp_2_weeks_ago, chain.value)  # Block 2 weeks ago
    )
    print(f"Collecting bpt prices for {chain.value}")
    pools = CORE_POOLS.get(chain.value, None)
    if pools is None:
        continue
    for core_pool in pools:
        bpt_twap_prices[chain.value][core_pool] = get_twap_bpt_price(
            core_pool, chain.value, getattr(WEB3_INSTANCES, chain.value),
            start_date=datetime.datetime.fromtimestamp(timestamp_now), block_number=target_blocks[chain.value][0]
        )

KeyError: 'ALCHEMY_KEY'

## Fetching fees and other data from the Balancer subgraphs

In [ ]:
from fee_allocator.config_file import BALANCER_GRAPH_URLS
from fee_allocator.helpers import get_balancer_pool_snapshots
from typing import Dict

pool_snapshots = {}
for chain in Chains:
    pool_snapshots[chain.value] = (
        get_balancer_pool_snapshots(target_blocks[chain.value][0], BALANCER_GRAPH_URLS[chain.value]),  # now
        get_balancer_pool_snapshots(target_blocks[chain.value][1], BALANCER_GRAPH_URLS[chain.value]),  # 2 weeks ago
    )

## Extract fee data for CORE pools:


In [ ]:
from collections import defaultdict
from decimal import Decimal
from fee_allocator.helpers import fetch_token_price_balgql


def collect_fee_info(pools: list[str], chain: Chains, pools_now: list[dict], pools_shifted: list[dict],
                     start_date: datetime.datetime) -> Dict[str, Dict]:
    """
    Collects fee info for all pools in the list. Returns dictionary with pool id as key and fee info as value 
    """
    fees = {}
    token_fees = defaultdict(list)
    for pool in pools:
        current_fees_snapshots = [x for x in pools_now if x['pool']['id'] == pool]
        current_fees_snapshots.sort(key=lambda x: x['timestamp'], reverse=True)
        fees_2_weeks_ago = [x for x in pools_shifted if x['pool']['id'] == pool]
        fees_2_weeks_ago.sort(key=lambda x: x['timestamp'], reverse=True)
        # If pools doesn't have current fees it means it was not created yet, so we skip it
        if not current_fees_snapshots:
            continue
        pool_snapshot_now = current_fees_snapshots[0]
        pool_snapshot_2_weeks_ago = fees_2_weeks_ago[0]
        # Now we need to collect token fee info. Let's start with BPT tokens, which is Balancer pool token. Notice,
        # That totalProtocolFeePaidInBPT can be null, so we need to check for that
        bpt_token_fee = 0
        token_fees_in_usd = 0
        bpt_price_usd = bpt_twap_prices[chain.value][pool] or 0
        if pool_snapshot_now['pool']['totalProtocolFeePaidInBPT']:
            bpt_token_fee = float(pool_snapshot_now['pool']['totalProtocolFeePaidInBPT']) - float(
                pool_snapshot_2_weeks_ago['pool']['totalProtocolFeePaidInBPT'] or 0)  # If 2 weeks ago is null, set to 0
        else:
            # Collect fee info about fees paid in pool tokens. Pool tokens fee info is in pool.tokens dictionary. This will be separate dictionary
            for token_data in pool_snapshot_now['pool']['tokens']:
                token_data_2_weeks_ago = \
                    [t for t in pool_snapshot_2_weeks_ago['pool']['tokens'] if t['address'] == token_data['address']][0]
                token_fee = float(token_data.get('paidProtocolFees', None)) - float(
                    token_data_2_weeks_ago.get('paidProtocolFees', None) or 0)
                # Get twap token price from Balancer API
                token_price = fetch_token_price_balgql(token_data['address'], chain.value, start_date) or 0
                token_fees_in_usd += Decimal(token_fee) * Decimal(token_price)
        fees[pool_snapshot_now['pool']['id']] = {
            'symbol': pool_snapshot_now['pool']['symbol'],
            'pool_addr': pool_snapshot_now['pool']['address'],
            'bpt_token_fee': round(bpt_token_fee, 2),
            # One of two fields below should always be 0 because fees are taken in either BPT or pool tokens
            'bpt_token_fee_in_usd': round(Decimal(bpt_token_fee) * bpt_price_usd, 2),
            'token_fees_in_usd': round(token_fees_in_usd, 2),
            'chain': chain.value,
            'token_fees': token_fees[pool_snapshot_now['pool']['symbol']]
        }
    return fees


collected_fees = {}

for chain in Chains:
    core_pools = CORE_POOLS.get(chain.value, None)
    if not core_pools:
        continue
    collected_fees[chain.value] = collect_fee_info(core_pools, chain, pool_snapshots[chain.value][0],
                                                   pool_snapshots[chain.value][1], datetime_now)
# Convert to dataframe, sort by chain and pool fee
joint_fees = {**collected_fees[Chains.MAINNET.value], **collected_fees[Chains.ARBITRUM.value],
              **collected_fees[Chains.POLYGON.value], **collected_fees[Chains.BASE.value],
              **collected_fees[Chains.AVALANCHE.value], **collected_fees.get(Chains.GNOSIS.value, {})}
joint_fees_df = pd.DataFrame.from_dict(joint_fees, orient='index')

## Now let's calculate incentives paid to the pools and save them to csv

In [ ]:
from fee_allocator.helpers import calculate_aura_vebal_share
import json

aura_vebal_share = calculate_aura_vebal_share(WEB3_INSTANCES.mainnet, target_blocks[Chains.MAINNET.value][0])

# Incentives are split per chain and per pool, with each pool getting a share of the incentive proportional to its share of fees
# paid by all pools on that chain. For example, if pool A paid 10% of all fees on Arbitrum, it will get 10% of the incentives. That 10% will be distributed between aura and vebal, proportional to their share of the incentive.

# Load fees from the json file
with open(f'../fees_collected/fees_{datetime_2_weeks_ago.date()}_{datetime_now.date()}.json') as f:
    FEES_TO_DISTRIBUTE = json.load(f)

# Let's calculate share of fees paid by each pool on each chain
def calc_and_split_fees(fees: Dict, chain: str, fees_to_distribute: Decimal) -> (Dict[str, Dict], list, Decimal):
    incentives_to_redirect = 0
    pool_incentives = {}
    no_incentive_pools = []
    # Calculate pool share in fees
    dao_share = Decimal(FEES_CONSTANTS['dao_share_pct'])
    vebal_share = Decimal(FEES_CONSTANTS['vebal_share_pct'])
    fees_to_distr_wo_dao_vebal = fees_to_distribute - (fees_to_distribute * dao_share) - (
            fees_to_distribute * vebal_share)
    # Calculate totals
    bpt_fees = sum([data['bpt_token_fee_in_usd'] for pool, data in fees.items()])
    token_fees = sum([data['token_fees_in_usd'] for pool, data in fees.items()])
    total_fees = bpt_fees + token_fees
    if not total_fees:
        return {}
    aura_bal_switch = True
    ### First pass to distribute incentives
    redirected_incentives= Decimal(0)
    for pool, data in fees.items():
        pool_fees = data['bpt_token_fee_in_usd'] + data['token_fees_in_usd']
        pool_share = pool_fees / Decimal(total_fees)
        # If aura incentives is less than 500 USDC, we pay all incentives to balancer
        total_incentives = pool_share * fees_to_distr_wo_dao_vebal
        if total_incentives < MIN_INCENTIVE:
            incentives_to_redirect += total_incentives
            no_incentive_pools.append(pool)
            total_incentives = 0
            redirected_incentives -= total_incentives
        fees_to_dao = round(pool_share * fees_to_distribute * dao_share, 2)
        fees_to_vebal = round(pool_share * fees_to_distribute * vebal_share, 2)
        
        # Dump results
        pool_incentives[pool] = {
            "chain": chain,
            "symbol": data['symbol'],
            "protocol_fees_collected": pool_fees,
            "fees_to_vebal": fees_to_vebal,
            "fees_to_dao": fees_to_dao,
            "total_incentives": total_incentives,
            "redirected_incentives": redirected_incentives,
            "reroute_incentives": Decimal(0),
            "pool_addr": data['pool_addr'],
        }
    return pool_incentives, no_incentive_pools, incentives_to_redirect

def calc_and_split_incentives(fees: Dict):
    for pool, data in fees.items():
        total_incentives = data['total_incentives']
        aura_incentives = round(total_incentives * aura_vebal_share, 2)
        if total_incentives == 0:
            continue
        if aura_incentives <= Decimal(MIN_AURA_INCENTIVE):
            if aura_bal_switch:
                aura_incentives = Decimal(0)
                bal_incentives = total_incentives
                aura_bal_switch = not aura_bal_switch
            else:
                aura_incentives = total_incentives
                bal_incentives = Decimal(0)
                aura_bal_switch = not aura_bal_switch

        else:
            bal_incentives = round(total_incentives * (1 - aura_vebal_share), 2)
            aura_incentives = total_incentives -bal_incentives
        fees[pool]['aura_incentives'] = aura_incentives
        fees[pool]['bal_incentives'] = bal_incentives

def re_route_incentives(incentives_data: Dict[str, Dict], chain: Chains) -> Dict[str, Dict]:
    """
    If pool is in re-route configuration, all incentives from that pool should be distributed to destination pool
      Ex: {source_pool: destination_pool}
    """
    if not chain.value in REROUTE_CONFIG:
        return incentives_data
    for pool_id, _data in incentives_data.items():
        if pool_id in REROUTE_CONFIG[chain.value]:
            # Re route everything to destination pool and set source pool incentives to 0
            incentives_data[REROUTE_CONFIG[chain.value][pool_id]]['aura_incentives'] += _data['aura_incentives']
            incentives_data[REROUTE_CONFIG[chain.value][pool_id]]['bal_incentives'] += _data['bal_incentives']
            # Increase total incentives by aura and bal incentives
            incentives_data[REROUTE_CONFIG[chain.value][pool_id]]['total_incentives'] += _data['aura_incentives'] + \
                                                                                         _data[                                                                                             'bal_incentives']
            # Mark source pool incentives as rerouted
            incentives_data[REROUTE_CONFIG[chain.value][pool_id]]['reroute_incentives'] += _data['total_incentives']
            incentives_data[pool_id]['aura_incentives'] = 0
            incentives_data[pool_id]['bal_incentives'] = 0
    return incentives_data


def re_distribute_incentives(incentives_data: Dict[str, Dict], chain: Chains, no_incentive_pools: list, amount_to_redirect: int) -> Dict[str, Dict]:
    """
    If some pools received < min_vote_incentive_amount all incentives from that pool
        should be evenly distributed between all pools that received > min_vote_incentive_amount
    """
    # Gather information about what pools need to be redistributed to and work out a total to scale them by.
    all_pools = set(incentives_data.keys())
    recipient_pools = all_pools.symmetric_difference(set(no_incentive_pools))
    new_grand_total = 0
    for pool in recipient_pools:
        assert incentives_data[pool]['chain'] == chain, "Chains in incentive data doesn't match working chain"
        new_grand_total += pool['protocol_fees_collected']    
    for pool_id in recipient_pools:
        _data = incentives_data[pool_id]
        # Calculate incentives to receive
        incentives_data[pool_id]['total_incentives'] += round(amount_to_redirect * (incentives_data[pool_id]['total_incentives']/new_grand_total), 2)
        incentives_data[pool_id]['redirected_incentives'] +=  round(amount_to_redirect * (incentives_data[pool_id]['total_incentives']/new_grand_total), 2)


incentives = {}
for chain in Chains:
    if not collected_fees.get(chain.value):
        continue
    (_incentives, no_incentive_pools, incentives_to_redirect) = calc_and_split_fees(collected_fees[chain.value], chain.value,
                                            Decimal(FEES_TO_DISTRIBUTE[chain.value]))
    # Reroute incentives if needed
    re_routed_incentives = re_route_incentives(_incentives, chain)
    split_incentives = calc_and_split_incentives(re_routed_incentives, chain)
    incentives[chain.value] = re_distribute_incentives(calc_and_split_incentives(), chain, no_incentive_pools, incentives_to_redirect)
    
joint_incentives_data = {**incentives[Chains.MAINNET.value], **incentives[Chains.ARBITRUM.value],
                         **incentives[Chains.POLYGON.value], **incentives[Chains.BASE.value],
                         **incentives[Chains.AVALANCHE.value]}

joint_incentives_data = {k: v for k, v in sorted(joint_incentives_data.items(), key=lambda item: item[1]['chain'])}
joint_incentives_df = pd.DataFrame.from_dict(joint_incentives_data, orient='index')
# Don't show pool_addr
display = joint_incentives_df.drop(columns=['pool_addr'])
display_sorted = display.sort_values(by=['chain', 'protocol_fees_collected'], ascending=False)
display_sorted.to_csv(f'../allocations/incentives_{datetime_2_weeks_ago.date()}_{datetime_now.date()}.csv')
display_sorted

## Reconcile

In [ ]:
all_fees_sum = Decimal(round(sum(FEES_TO_DISTRIBUTE.values()), 2))
# Sum of fees_to_vebal + fees_to_dao + aura_incentives + bal_incentives
all_incentives_sum = sum([sum([x['fees_to_vebal'], x['fees_to_dao'], x['aura_incentives'], x['bal_incentives']]) for x in
                          joint_incentives_data.values()])
# Sum each column separately using pandas and create a new dataframe
reconcile_df = pd.DataFrame(
    {
        'fees_to_distribute': [all_fees_sum],
        'fees_to_vebal': [sum([x['fees_to_vebal'] for x in joint_incentives_data.values()])],
        'fees_to_dao': [sum([x['fees_to_dao'] for x in joint_incentives_data.values()])],
        'bal_incentives': [sum([x['bal_incentives'] for x in joint_incentives_data.values()])],
        'aura_incentives': [sum([x['aura_incentives'] for x in joint_incentives_data.values()])],
        'total_incentives': [all_incentives_sum],
        # Reconcile total incentives
        'delta': [all_fees_sum - all_incentives_sum],
        # SUM should be 0 or close to 0
        'redirected(should be around 0)': [sum([x['redirected_incentives'] for x in joint_incentives_data.values()])],
    }
)
reconcile_df